In [1]:
import os
import json

In [2]:
# 파일 위치 지정
keys_file_path = os.path.join('data', 'api_keys.txt')

# 파일에서 API 키를 로드하는 함수
def load_api_keys(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        keys = json.load(file)
    return keys

# API 키 사용
api_keys = load_api_keys(keys_file_path)
openAI_keys = api_keys['openAI_keys']
pinecone_keys = api_keys['pinecone_keys']

In [3]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 연결 및 index 설정
pc = Pinecone(api_key=pinecone_keys)
index = pc.Index('rag')

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 6e-05,
 'namespaces': {'': {'vector_count': 6}},
 'total_vector_count': 6}

In [4]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    temperature=0,
    max_tokens=2048,
    model_name="gpt-3.5-turbo",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [5]:
from langchain.prompts import PromptTemplate

template = """Information: {retrieval}
Question: {query}"""

prompt = PromptTemplate.from_template(template=template)

In [6]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [7]:
query = "차는 어느 나라에서 주로 마시지?"

In [8]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input = query,
    model="text-embedding-3-small"
)

In [9]:
matches = index.query(
    vector=response.data[0].embedding,
    top_k=3,
    include_values=False,
    include_metadata=True,
)

In [10]:
# 찾아온 벡터들의 토큰화된 metadata들 다시 이어 붙임
assertions_list = [match['metadata']['assertion'].strip() for match in matches['matches']]

print(assertions_list)

['The Citrine Wagtail is seen throughout Europe and Central Asia.', 'The Citrine Wagtail is seen throughout Europe and Central Asia.', 'Central Asia is the land where fermented milk really came into vogue.']


In [11]:
result = llm_chain.invoke({"retrieval": assertions_list, "query":query})

Answer: 중앙 아시아에서 발효 우유가 유행했습니다.